In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2022-10-07T17:24:46.249751-04:00

Python implementation: CPython
Python version       : 3.8.9
IPython version      : 8.4.0

Compiler    : Clang 13.1.6 (clang-1316.0.21.2.5)
OS          : Darwin
Release     : 21.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 20
Architecture: 64bit



In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score
import sklearn
import gc
import os

In [4]:
%watermark --iversions

numpy  : 1.23.1
pandas : 1.4.3
sklearn: 1.1.2



In [5]:
train = pd.read_csv('../input/train_loan.csv.gz')
test = pd.read_csv('../input/test_loan.csv.gz')

In [6]:
train.head()

feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0   0.833333        0.0        0.4   0.005263   0.005185   0.185751   
1   1.000000        0.0        1.0   0.003474   0.023804   0.134860   
2   0.666667        0.0        0.8   0.005263   0.029700   0.236641   
3   0.333333        0.0        0.5   0.007158   0.024434   0.745547   
4   0.666667        0.0        0.7   0.006842   0.021962   0.440204   

   feature_6  feature_7  feature_8  feature_9  ...  feature_19  feature_20  \
0   0.240654        0.0        0.0        0.0  ...         0.0         0.0   
1   0.036215        0.0        0.0        0.0  ...         0.0         0.0   
2   0.271807        0.0        0.0        1.0  ...         0.0         0.0   
3   0.778037        1.0        0.0        0.0  ...         0.0         0.0   
4   0.260125        0.0        0.0        1.0  ...         0.0         0.0   

   feature_21  feature_22  feature_23  feature_24  feature_25  feature_26  \
0         0.0         0.0         0.0         0.0         0.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         0.0   
3         0.0         0.0         0.0         0.0         0.0         0.0   
4         0.0         0.0         0.0         0.0         0.0         0.0   

   feature_27  ground_truth  
0         0.0             0  
1         0.0             0  
2         0.0             0  
3         0.0             0  
4         0.0             0  

[5 rows x 29 columns]

In [7]:
test.head()

feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0   0.500000   0.000000        0.0   0.008632   0.024274   0.541985   
1   0.833333   0.083333        0.3   0.007263   0.004775   0.211196   
2   0.833333   0.000000        1.0   0.011579   0.014474   0.211196   
3   0.500000   0.000000        1.0   0.004316   0.029600   0.363868   
4   0.500000   0.083333        0.2   0.009737   0.012432   0.236641   

   feature_6  feature_7  feature_8  feature_9  ...  feature_19  feature_20  \
0   0.488707        1.0        0.0        1.0  ...         0.0         0.0   
1   0.247664        0.0        0.0        0.0  ...         0.0         0.0   
2   0.211838        0.0        0.0        1.0  ...         0.0         0.0   
3   0.437695        1.0        0.0        1.0  ...         0.0         0.0   
4   0.572040        0.0        0.0        1.0  ...         0.0         0.0   

   feature_21  feature_22  feature_23  feature_24  feature_25  feature_26  \
0         0.0         0.0         0.0         0.0         0.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         0.0   
3         0.0         0.0         0.0         0.0         0.0         0.0   
4         0.0         0.0         0.0         0.0         0.0         0.0   

   feature_27  ground_truth  
0         0.0             1  
1         0.0             0  
2         0.0             0  
3         0.0             1  
4         0.0             1  

[5 rows x 29 columns]

In [7]:
X = train[train.columns[:-1]].values
X_test = test[test.columns[:-1]].values
Y = train.ground_truth.values
Y_test = test.ground_truth.values

In [8]:
np.mean(Y)

0.4932015275963286

In [9]:
train_oof = np.zeros((X.shape[0], ))
test_preds = 0
train_oof.shape

(418697,)

In [ ]:
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = SVC(probability=True)
    model.fit(train_features, train_target)
    val_pred = model.predict(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, val_pred))
    test_preds += model.predict_proba(X_test)[:,1]/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1


In [16]:
train_oof = np.zeros((X.shape[0], ))
test_preds = 0

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = LogisticRegression(C=5, max_iter=90)
    model.fit(train_features, train_target)
    val_pred = model.predict(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, val_pred))
    test_preds += model.predict_proba(X_test)[:,1]/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1


/Users/tunguz/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold accuracy: 0.6445545736804394
Fitting fold 2


/Users/tunguz/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold accuracy: 0.6461069978504896
Fitting fold 3


/Users/tunguz/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold accuracy: 0.6450638292790695
Fitting fold 4


/Users/tunguz/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold accuracy: 0.6467476325248689
Fitting fold 5
Fold accuracy: 0.6477268656181707


/Users/tunguz/Library/Python/3.8/lib/python/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
